# Jupyter Interactive Environment

In [ ]:
# custom module
import topodisc as td

import numpy as np
import pandas as pd

## Map Data

In [ ]:
# on my machine
map_path = "../data/map/"

centers = pd.read_csv(f'{map_path}centers.csv').set_index('cID')
samples = pd.read_csv(f'{map_path}samples.csv').set_index('sID')

samples['DISC'] = samples.apply(
    lambda row: ((row['AZ2'] - row['AZ1'] + 180) % 360) - 180,
    axis=1
)

cIDs = centers.index.tolist()
sIDs = samples.index.tolist()

POP_A_sIDs = [
    198, 438, 439, 440, 441, 442, 443, 444, 445, 490, 491, 492, 493
]

POP_B_sIDs = [
    193, 194, 195, 196, 197, 477, 478, 479, 480, 481, 544, 545, 548, 549
]

POP_C_sIDs = [
    529, 530, 531, 532, 533, 537, 538, 539, 540, 541, 542, 543
]

POP_ABC_sIDs = POP_A_sIDs + POP_B_sIDs + POP_C_sIDs

pop_all = td.Population(name='all', sIDs=sIDs)
pop_abc = td.Population(name='ABC', sIDs=POP_ABC_sIDs)

Calculate tilt (etc.) for each sample, for each center.

In [ ]:
paleo_azimuth_uncertainty = 0

centers_calc = {
    cID: td.make_center(cID, centers, samples, paleo_azimuth_uncertainty)
    for cID in cIDs
}

Define criteria and perform evaluation. This takes less than two minutes on my machine.

In [ ]:
min_offset = 7

criteria = [
    td.Criterion(td.subset_sizes(min_offset), pop_abc),
    td.Criterion(td.inflation_score(min_offset), pop_abc),
    td.Criterion(td.subset_sizes(min_offset), pop_all),
    td.Criterion(td.inflation_score(min_offset), pop_all),
]

scores = {}
score_names = []

for best_cID, center in centers_calc.items():
    scores_for_this_center = []
    for crit in criteria:
        scores_for_this_criterion = td.evaluate_center(center, crit)
        for key, val in scores_for_this_criterion.items():
            scores_for_this_center.append(val)
            full_score_name = f'{crit.pop.name}_{key}'
            if full_score_name not in score_names:
                score_names.append(full_score_name)
    scores[best_cID] = scores_for_this_center

centers_eval = pd.DataFrame(scores).transpose().set_axis(score_names, axis=1)
centers_eval_no_infs = centers_eval.replace([np.inf, -np.inf], np.nan)

Write evaluation to disk.

In [ ]:
centers_eval_no_infs.to_csv(
    f'{map_path}centers_eval.csv',
    index_label='cID'
)

## Model data

A non-exhaustive set of parametric sweeps previously implemented in COMSOL. I arrived at the combinations shown here after several iterations of searching through increasingly fine resolution sweeps. 

In [ ]:
shallow_params = {
    "depth": [6000, 7000, 8000, 9000, 10_000],
    "radius": [25_000, 30_000, 35_000],
    "aspect": [0.04, 0.06, 0.08, 0.1, 0.12],
    "pmult": [-.2, -.4, -.6, -.8],
    "grav": [False],
    "topo": [True],
}

medium_params = {
    "depth": [10_000, 12_000, 14_000, 16_000, 18_000],
    "radius": [35_000, 40_000, 45_000],
    "aspect": [0.04, 0.06, 0.08, 0.1, 0.12],
    "pmult": [-.2, -.4, -.6, -.8],
    "grav": [False],
    "topo": [True],
}

deep_params = {
    "depth": [20_000, 21_000, 22_000, 23_000],
    "radius": [44_000, 45_000, 46_000, 47_000, 48_000, 49_000, 50_000],
    "aspect": [0.04, 0.06, 0.08, 0.1, 0.12],
    "pmult": [-1],
    "grav": [False],
    "topo": [True],
}

Calculate numerical tilt solution for every combination (from numerical displacement results).

In [ ]:
# on my machine
model_path = "../data/model/"

# paleo-edifice spline data
model_topo = np.genfromtxt(f'{model_path}z1.csv', delimiter=",").T

shallow_sweep = td.ParamSweep(
    td.unpack_param_combinations(shallow_params), model_topo
)

medium_sweep = td.ParamSweep(
    td.unpack_param_combinations(medium_params), model_topo
)

deep_sweep = td.ParamSweep(
    td.unpack_param_combinations(deep_params),
    model_topo
)

For each parametric sweep, sort the associated numerical solutions by RMSE fit to the tilt-distance dataset associated with the best inflation center candidate.

In [ ]:
best_cID = 486

shallow_sweep.sort_models_by_rmse(
    center=centers_calc[best_cID],
    sIDs=pop_abc.sIDs
    )

medium_sweep.sort_models_by_rmse(
    center=centers_calc[best_cID],
    sIDs=pop_abc.sIDs
    )

deep_sweep.sort_models_by_rmse(
    center=centers_calc[best_cID],
    sIDs=pop_abc.sIDs
    )

Print the best fit from each sweep, which is first after being sorted by RMSE.

In [ ]:
print(shallow_sweep.models[0])
print(medium_sweep.models[0])
print(deep_sweep.models[0])